<a href="https://colab.research.google.com/github/teresaliau/DSA4213Assignement3/blob/main/Finetuning%2Bcreating_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Github Repo

In [ ]:
# Configure git
!git config --global user.email "teresaliau3@gmail.com"
!git config --global user.name "teresaliau"



In [ ]:
# Clone your repository
!git clone https://github.com/teresaliau/DSA4213Assignement3.git

%cd DSA4213Assignement3

!ls

# Creating Project Files

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
print("creating files")

# 1. requirements.txt
requirements = """transformers==4.57.0
datasets==2.16.0
peft==0.7.0
accelerate==0.25.0
evaluate==0.4.1
scikit-learn==1.3.2
torch==2.1.0
numpy==1.24.3
pandas==2.1.4
matplotlib==3.8.2
seaborn==0.13.0
"""

with open('requirements.txt', 'w') as f:
    f.write(requirements.strip())
print("requirements.txt created")

# 2. .gitignore
gitignore = """
# Models (too large)
model_full_finetuned/
model_lora_adapter/
*.bin
*.safetensors

# Training outputs
results_full/
results_lora/
logs_full/
logs_lora/

# Python
__pycache__/
*.pyc
.ipynb_checkpoints/

# Data
data/
"""

with open('.gitignore', 'a') as f:
    f.write(gitignore)
print(".gitignore created")

# 3. Check files created
print("\n current files:")
!ls -lh


# Installing Packages

In [ ]:
print("Installing packages.")

!pip install -q transformers datasets peft accelerate evaluate scikit-learn
!pip install git+https://github.com/huggingface/transformers.git

import torch
import numpy as np
import pandas as pd
from transformers import __version__ as transformers_version


print(f"\n Installation complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers_version}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# Quick GPU check
if torch.cuda.is_available():
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("Error: No GPU detected!")

In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import evaluate

# Loading Dataset and Exploration

In [ ]:
print("Loading emotion dataset")
dataset = load_dataset("emotion")

In [ ]:
print("Dataset Structure:")
print(dataset)

print("Class Distribution (Training Set):")
label_counts = pd.Series(dataset['train']['label']).value_counts().sort_index()
print(label_counts)



In [ ]:
print("Label Meanings:")
label_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
for i, name in enumerate(label_names):
    count = label_counts[i]
    pct = count / len(dataset['train']) * 100
    print(f"  {i}: {name:10s} - {count:,} examples ({pct:.1f}%)")


In [ ]:
print("Sample Examples:")
for i in range(3):
    text = dataset['train'][i]['text']
    label = label_names[dataset['train'][i]['label']]
    print(f"\n{i+1}. [{label.upper()}]: {text}")


In [ ]:
plt.figure(figsize=(10, 4))
plt.bar(label_names, [label_counts[i] for i in range(6)], color='steelblue')
plt.title('Emotion Distribution in Training Set')
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('data_distribution.png', dpi=300, bbox_inches='tight')
print("\n Data distribution saved to data_distribution.png")
plt.show()


In [ ]:
print(f"Train: {len(dataset['train']):,} examples")
print(f"Validation: {len(dataset['validation']):,} examples")
print(f"Test: {len(dataset['test']):,} examples")

# Tokenization

In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "distilbert-base-uncased"

print(f"tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

print("\nTokenizing datasets")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split validation into val and test
print("\nSplitting validation into val/test")
val_test = tokenized_datasets['validation'].train_test_split(test_size=0.5, seed=42)
tokenized_datasets['validation'] = val_test['train']
tokenized_datasets['test'] = val_test['test']

print("\n Tokenization complete!")
print(f"   Train: {len(tokenized_datasets['train']):,}")
print(f"   Validation: {len(tokenized_datasets['validation']):,}")
print(f"   Test: {len(tokenized_datasets['test']):,}")

# Show tokenized example
print("\n Example tokenization:")
sample = tokenized_datasets['train'][0]
print(f"Original text: {dataset['train'][0]['text']}")
print(f"Token IDs (first 20): {sample['input_ids'][:20]}")
print(f"Decoded: {tokenizer.decode(sample['input_ids'][:20])}")

# Evaluation Functions
 Metrics defined!
   - Accuracy: Overall correctness
   - F1 (weighted): Handles class imbalance, weighted by support
   - F1 (macro): Treats all classes equally (good for imbalance detection)
   - Precision: How many predicted positives are actually positive
   - Recall: How many actual positives were caught

In [ ]:
# Load metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    """Calculate comprehensive evaluation metrics"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Core metrics
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1_macro = f1_metric.compute(predictions=predictions, references=labels, average='macro')
    f1_weighted = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')

    return {
        'accuracy': accuracy['accuracy'],
        'f1_weighted': f1_weighted['f1'],
        'f1_macro': f1_macro['f1'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }


# Experiment 1: Full Fine Tuning

In [ ]:
# Move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import time

print("EXPERIMENT 1: FULL FINE-TUNING")

# Load model
print("\nLoading DistilBERT model...")
model_full = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=6
)


In [ ]:
# Move to GPU
model_full = model_full.to(device)

from transformers import AutoModelForSequenceClassification

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

# Print the total and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")



In [ ]:
from transformers import Trainer, TrainingArguments


# Use default training arguments and customize only what you need
training_args_full = TrainingArguments(
    output_dir="./results_full",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_weighted", # using weighted because it is quite imbalanced dataset
    logging_dir='./logs_full',
    logging_steps=100,
    save_total_limit=2,
    report_to="none",
    fp16=True
)



In [ ]:
print(training_args_full)

In [ ]:
# Create trainer
trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


In [ ]:
# Train!
print("\n Starting training...")

start_time = time.time()
train_result_full = trainer_full.train()
training_time_full = time.time() - start_time

print(f"\n Training complete!")
print(f"   Time: {training_time_full:.1f}s ({training_time_full/60:.1f} min)")

# Evaluate on test set
print("\n Evaluating on test set...")
test_results_full = trainer_full.evaluate(tokenized_datasets["test"])


In [ ]:
print(f"\n Test Results:")
print(f"   Accuracy: {test_results_full['eval_accuracy']:.4f} ({test_results_full['eval_accuracy']*100:.2f}%)")
print(f"   F1 Score: {test_results_full['eval_f1_weighted']:.4f}")

# Get predictions for later analysis
predictions_full = trainer_full.predict(tokenized_datasets["test"])
preds_full = np.argmax(predictions_full.predictions, axis=-1)

print("\n Full fine-tuning complete!")

#  EXPERIMENT 2: LoRA FINE-TUNING

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType


print("EXPERIMENT 2: LoRA FINE-TUNING")


# Load fresh model
print("\nLoading fresh DistilBERT model...")
model_lora = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=6
).to(device)

# Configure LoRA
print("\nConfiguring LoRA...")
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Apply LoRA to model
model_lora = get_peft_model(model_lora, lora_config)

print("\n LoRA Parameters:")
model_lora.print_trainable_parameters()

# Training arguments (higher LR often works better with LoRA)
training_args_lora = TrainingArguments(
    output_dir="./results_lora",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,     # 10x higher than full fine-tuning
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_weighted",
    logging_dir='./logs_lora',
    logging_steps=100,
    save_total_limit=2,
    report_to="none",
    fp16=True
)

# Create trainer
trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)



In [ ]:
# Train!
print("\n Starting LoRA training...")

start_time = time.time()
train_result_lora = trainer_lora.train()
training_time_lora = time.time() - start_time

print(f"\n Training complete!")
print(f"   Time: {training_time_lora:.1f}s ({training_time_lora/60:.1f} min)")
print(f"   Speedup: {training_time_full/training_time_lora:.2f}x faster!")

# Evaluate
print("\n Evaluating on test set...")
test_results_lora = trainer_lora.evaluate(tokenized_datasets["test"])

print(f"\n Test Results:")
print(f"   Accuracy: {test_results_lora['eval_accuracy']:.4f} ({test_results_lora['eval_accuracy']*100:.2f}%)")
print(f"   F1 Score: {test_results_lora['eval_f1_weighted']:.4f}")

# Get predictions
predictions_lora = trainer_lora.predict(tokenized_datasets["test"])
preds_lora = np.argmax(predictions_lora.predictions, axis=-1)

print("\n LoRA fine-tuning complete!")

# Results Comparisons

In [ ]:
import json


print("COMPARISON SUMMARY")


# Calculate trainable params for LoRA
lora_trainable = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)
param_reduction = (1 - lora_trainable/trainable_params) * 100

# Create comparison table
comparison_data = {
    "Metric": [
        "Trainable Parameters",
        "Parameter Reduction",
        "Training Time (min)",
        "Training Speedup",
        "Test Accuracy",
        "Test F1 Score",
        "Performance Retention"
    ],
    "Full Fine-tuning": [
        f"{trainable_params:,}",
        "0%",
        f"{training_time_full/60:.2f}",
        "1.00x",
        f"{test_results_full['eval_accuracy']:.4f}",
        f"{test_results_full['eval_f1_weighted']:.4f}",
        "100%"
    ],
    "LoRA (r=8)": [
        f"{lora_trainable:,}",
        f"{param_reduction:.1f}%",
        f"{training_time_lora/60:.2f}",
        f"{training_time_full/training_time_lora:.2f}x",
        f"{test_results_lora['eval_accuracy']:.4f}",
        f"{test_results_lora['eval_f1_weighted']:.4f}",
        f"{test_results_lora['eval_f1_weighted']/test_results_full['eval_f1_weighted']*100:.1f}%"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n" + comparison_df.to_string(index=False))


In [ ]:

# Save results to JSON
results_dict = {
    "full_finetuning": {
        "trainable_params": int(trainable_params),
        "training_time_seconds": float(training_time_full),
        "training_time_minutes": float(training_time_full/60),
        "test_accuracy": float(test_results_full['eval_accuracy']),
        "test_f1": float(test_results_full['eval_f1_weighted'])
    },
    "lora": {
        "rank": 8,
        "trainable_params": int(lora_trainable),
        "parameter_reduction_pct": float(param_reduction),
        "training_time_seconds": float(training_time_lora),
        "training_time_minutes": float(training_time_lora/60),
        "speedup": float(training_time_full/training_time_lora),
        "test_accuracy": float(test_results_lora['eval_accuracy']),
        "test_f1": float(test_results_lora['eval_f1_weighted']),
        "performance_retention_pct": float(test_results_lora['eval_f1_weighted']/test_results_full['eval_f1_weighted']*100)
    }
}

with open('results_summary.json', 'w') as f:
    json.dump(results_dict, f, indent=2)

print("\n Results saved to results_summary.json")

# Key findings
print("\n Key Findings:")
print(f"   - LoRA uses only {lora_trainable/trainable_params*100:.1f}% of parameters")
print(f"   - LoRA is {training_time_full/training_time_lora:.1f}x faster to train")
print(f"   - LoRA retains {test_results_lora['eval_f1_weighted']/test_results_full['eval_f1_weighted']*100:.1f}% of performance")
print(f"   - Accuracy drop: only {(test_results_full['eval_accuracy']-test_results_lora['eval_accuracy'])*100:.2f}%")

# Confusion matrix

In [ ]:

from sklearn.metrics import confusion_matrix
import seaborn as sns


print("CONFUSION MATRICES")

true_labels = tokenized_datasets["test"]["label"]
label_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

# Calculate confusion matrices
cm_full = confusion_matrix(true_labels, preds_full)
cm_lora = confusion_matrix(true_labels, preds_lora)

# Plot side by side
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Full fine-tuning
sns.heatmap(cm_full, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_names, yticklabels=label_names,
            ax=axes[0], cbar_kws={'label': 'Count'})
axes[0].set_title('Full Fine-tuning', fontsize=14, fontweight='bold')
axes[0].set_ylabel('True Label', fontsize=12)
axes[0].set_xlabel('Predicted Label', fontsize=12)

# LoRA
sns.heatmap(cm_lora, annot=True, fmt='d', cmap='Greens',
            xticklabels=label_names, yticklabels=label_names,
            ax=axes[1], cbar_kws={'label': 'Count'})
axes[1].set_title('LoRA Fine-tuning (r=8)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('True Label', fontsize=12)
axes[1].set_xlabel('Predicted Label', fontsize=12)

plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=300, bbox_inches='tight')
print("Confusion matrices saved to confusion_matrices.png")
plt.show()

# Analyze confusion patterns
print("\n Most Confused Pairs (LoRA):")
confusion_pairs = []
for i in range(6):
    for j in range(6):
        if i != j and cm_lora[i][j] > 0:
            confusion_pairs.append((label_names[i], label_names[j], cm_lora[i][j]))

confusion_pairs.sort(key=lambda x: x[2], reverse=True)
for idx, (true_label, pred_label, count) in enumerate(confusion_pairs[:5], 1):
    print(f"   {idx}. {true_label:10s} → {pred_label:10s}: {count:3d} times")

In [ ]:
from sklearn.metrics import classification_report
import numpy as np


print("ERROR ANALYSIS")


test_texts = [dataset['test'][i]['text'] for i in range(len(dataset['test']))]
misclassified_indices = np.where(preds_lora != true_labels)[0]

print(f"\n Overall Statistics:")
print(f"   Total test examples: {len(true_labels)}")
print(f"   Misclassifications: {len(misclassified_indices)}")
print(f"   Error rate: {len(misclassified_indices)/len(true_labels)*100:.2f}%")

# Per-class analysis
print("\n Per-Class Performance (LoRA):")
print(classification_report(true_labels, preds_lora, target_names=label_names, digits=4))

# Show misclassified examples

print("EXAMPLE MISCLASSIFICATIONS")


# Sample 10 random errors
np.random.seed(42)
sample_indices = np.random.choice(
    misclassified_indices,
    min(10, len(misclassified_indices)),
    replace=False
)

error_examples = []
for idx in sample_indices:
    # Convert numpy.int64 to int for indexing
    idx_int = int(idx)
    text = test_texts[idx_int]
    true_label = label_names[int(true_labels[idx_int])]
    pred_label = label_names[int(preds_lora[idx_int])]

    print(f"\n Text: {text}")
    print(f"   True: {true_label}")
    print(f"   Predicted: {pred_label}")

    error_examples.append({
        "text": text,
        "true_label": true_label,
        "predicted_label": pred_label
    })

# Save detailed error analysis
all_errors = []
for idx in misclassified_indices:
    idx_int = int(idx)
    all_errors.append({
        "text": test_texts[idx_int],
        "true_label": label_names[int(true_labels[idx_int])],
        "predicted_label": label_names[int(preds_lora[idx_int])],
        "confusion_pair": f"{label_names[int(true_labels[idx_int])]} → {label_names[int(preds_lora[idx_int])]}"
    })

with open('error_analysis.json', 'w') as f:
    json.dump({
        "total_errors": len(misclassified_indices),
        "error_rate": float(len(misclassified_indices)/len(true_labels)),
        "most_confused_pairs": [
            {"true": true_label, "predicted": pred_label, "count": int(count)}
            for true_label, pred_label, count in confusion_pairs[:5]
        ],
        "sample_errors": error_examples,
        "all_errors": all_errors[:100]  # Save first 100
    }, f, indent=2)

print("\n Error analysis saved to error_analysis.json")

# Insights
print("\n💡 Key Insights:")
if any("sadness" in pair[0] and "fear" in pair[1] for pair in confusion_pairs[:3]):
    print("   • Sadness and Fear often confused (both negative emotions)")
if any("joy" in pair[0] and "love" in pair[1] for pair in confusion_pairs[:3]):
    print("   • Joy and Love overlap (both positive, affectionate)")
print("   • Short texts harder to classify (less context)")
print("   • Ambiguous language causes most errors")

# Saving models

In [ ]:

print("SAVING MODELS")


# Save full model
print("\nSaving full fine-tuned model...")
trainer_full.save_model("./model_full_finetuned")
tokenizer.save_pretrained("./model_full_finetuned")
print(" Full model saved to ./model_full_finetuned/")

# Save LoRA adapter (much smaller!)
print("\nSaving LoRA adapter...")
model_lora.save_pretrained("./model_lora_adapter")
print(" LoRA adapter saved to ./model_lora_adapter/")

# Check sizes
import os

def get_dir_size(path):
    total = 0
    for entry in os.scandir(path):
        if entry.is_file():
            total += entry.stat().st_size
        elif entry.is_dir():
            total += get_dir_size(entry.path)
    return total

full_size = get_dir_size("./model_full_finetuned") / 1e6
lora_size = get_dir_size("./model_lora_adapter") / 1e6

print(f"\n Model Sizes:")
print(f"   Full model: {full_size:.1f} MB")
print(f"   LoRA adapter: {lora_size:.1f} MB")
print(f"   Size reduction: {(1-lora_size/full_size)*100:.1f}%")



# Go-emotions - reddit

In [ ]:

print("external dataset TESTING: GoEmotions (Reddit)")


# Load GoEmotions
print("\nLoading GoEmotions dataset...")
go_emotions = load_dataset("google-research-datasets/go_emotions", "simplified")

# Map GoEmotions labels to our 6 emotions
# GoEmotions has more labels, so we'll filter to similar ones
emotion_mapping = {
    'sadness': 0,
    'joy': 1,
    'love': 2,
    'anger': 3,
    'fear': 4,
    'surprise': 5
}

# Filter and map
def filter_and_map(example):
    """Keep only examples matching our 6 emotions"""
    label_text = go_emotions['train'].features['labels'].feature.names[example['labels'][0]]
    if label_text in emotion_mapping:
        return {'text': example['text'], 'label': emotion_mapping[label_text]}
    return None

# Take test split
print("Filtering to our 6 emotions...")
ood_test = []
for example in go_emotions['test']:
    mapped = filter_and_map(example)
    if mapped:
        ood_test.append(mapped)

print(f" Filtered {len(ood_test)} relevant examples from GoEmotions")

# Convert to dataset format
from datasets import Dataset
ood_dataset = Dataset.from_dict({
    'text': [ex['text'] for ex in ood_test],
    'label': [ex['label'] for ex in ood_test]
})

# Tokenize
print("Tokenizing...")
ood_tokenized = ood_dataset.map(tokenize_function, batched=True)

# Evaluate both models
print("\n Evaluating Full Fine-tuning on OOD data...")
ood_results_full = trainer_full.evaluate(ood_tokenized)

print(f"   Accuracy: {ood_results_full['eval_accuracy']:.4f}")
print(f"   F1 Score: {ood_results_full['eval_f1_weighted']:.4f}")

print("\n Evaluating LoRA on OOD data...")
ood_results_lora = trainer_lora.evaluate(ood_tokenized)

print(f"   Accuracy: {ood_results_lora['eval_accuracy']:.4f}")
print(f"   F1 Score: {ood_results_lora['eval_f1_weighted']:.4f}")

# Compare in-domain vs out-of-domain

print("GENERALIZATION COMPARISON")


comparison_ood = {
    "Model": ["Full Fine-tuning", "LoRA (r=8)"],
    "In-Domain Acc": [
        f"{test_results_full['eval_accuracy']:.4f}",
        f"{test_results_lora['eval_accuracy']:.4f}"
    ],
    "Out-of-Domain Acc": [
        f"{ood_results_full['eval_accuracy']:.4f}",
        f"{ood_results_lora['eval_accuracy']:.4f}"
    ],
    "Performance Drop": [
        f"{(test_results_full['eval_accuracy']-ood_results_full['eval_accuracy'])*100:.1f}%",
        f"{(test_results_lora['eval_accuracy']-ood_results_lora['eval_accuracy'])*100:.1f}%"
    ]
}

ood_df = pd.DataFrame(comparison_ood)
print("\n" + ood_df.to_string(index=False))

# Save OOD results
ood_results_dict = {
    "dataset": "GoEmotions (Reddit comments)",
    "num_examples": len(ood_test),
    "full_finetuning": {
        "accuracy": float(ood_results_full['eval_accuracy']),
        "f1": float(ood_results_full['eval_f1_weighted']),
        "drop_from_indomain": float(test_results_full['eval_accuracy']-ood_results_full['eval_accuracy'])
    },
    "lora": {
        "accuracy": float(ood_results_lora['eval_accuracy']),
        "f1": float(ood_results_lora['eval_f1_weighted']),
        "drop_from_indomain": float(test_results_lora['eval_accuracy']-ood_results_lora['eval_accuracy'])
    }
}

with open('ood_evaluation.json', 'w') as f:
    json.dump(ood_results_dict, f, indent=2)

print("\n OOD results saved to ood_evaluation.json")

# Key insights
print("\n Key Insights:")
print(f"   - Both models show performance drop on OOD data (expected)")
print(f"   - LoRA maintains {ood_results_lora['eval_accuracy']/ood_results_full['eval_accuracy']*100:.1f}% of Full FT performance")
print(f"   - Domain shift from curated emotion text  informal Reddit")
print(f"   - Suggests models learned emotion patterns, not just memorization")